In [2]:
%pip install nvidia-smi


Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install --update accelerate
%pip uninstall -y transformers accelerate
%pip install transformers accelerate

Note: you may need to restart the kernel to use updated packages.



Usage:   
  c:\Users\admin\.conda\envs\nlp_text_env\python.exe -m pip install [options] <requirement specifier> [package-index-options] ...
  c:\Users\admin\.conda\envs\nlp_text_env\python.exe -m pip install [options] -r <requirements file> [package-index-options] ...
  c:\Users\admin\.conda\envs\nlp_text_env\python.exe -m pip install [options] [-e] <vcs project url> ...
  c:\Users\admin\.conda\envs\nlp_text_env\python.exe -m pip install [options] [-e] <local project path> ...
  c:\Users\admin\.conda\envs\nlp_text_env\python.exe -m pip install [options] <archive url/path> ...

no such option: --update


Found existing installation: transformers 4.41.2
Uninstalling transformers-4.41.2:
  Successfully uninstalled transformers-4.41.2
Found existing installation: accelerate 0.31.0
Uninstalling accelerate-0.31.0:
  Successfully uninstalled accelerate-0.31.0
Note: you may need to restart the kernel to use updated packages.
  Using cached transformers-4.41.2-py3-none-any.whl.metadata (43 kB)
  Using cached accelerate-0.31.0-py3-none-any.whl.metadata (19 kB)
Using cached transformers-4.41.2-py3-none-any.whl (9.1 MB)
Using cached accelerate-0.31.0-py3-none-any.whl (309 kB)
Note: you may need to restart the kernel to use updated packages.


In [5]:
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk
import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
from datasets import load_dataset, load_metric

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

nltk.download("punkt")


c:\Users\admin\.conda\envs\nlp_text_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

device = "cuda" if torch.cuda.is_available() else "cpu"

device

'cpu'

In [7]:
model_ckpt = "google/pegasus-cnn_dailymail"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
#download & unzip data

# !wget https://github.com/pratyuds/Text-Summarizer-project/raw/main/raw_data/summarizer-data.zip


'wget' is not recognized as an internal or external command,


operable program or batch file.


In [8]:
# !unzip summarizer-data.zip

Archive:  summarizer-data.zip
  inflating: samsum-test.csv         
  inflating: samsum-train.csv        
  inflating: samsum-validation.csv   
   creating: samsum_dataset/
 extracting: samsum_dataset/dataset_dict.json  
   creating: samsum_dataset/test/
  inflating: samsum_dataset/test/data-00000-of-00001.arrow  
  inflating: samsum_dataset/test/dataset_info.json  
  inflating: samsum_dataset/test/state.json  
   creating: samsum_dataset/train/
  inflating: samsum_dataset/train/data-00000-of-00001.arrow  
  inflating: samsum_dataset/train/dataset_info.json  
  inflating: samsum_dataset/train/state.json  
   creating: samsum_dataset/validation/
  inflating: samsum_dataset/validation/data-00000-of-00001.arrow  
  inflating: samsum_dataset/validation/dataset_info.json  
  inflating: samsum_dataset/validation/state.json  


In [11]:
import requests
import zipfile
import io

url = "https://github.com/pratyuds/Text-Summarizer-project/raw/main/raw_data/summarizer-data.zip"

# Download the zip file
response = requests.get(url)
zip_file = zipfile.ZipFile(io.BytesIO(response.content))

# Specify the directory where you want to extract the contents
extract_dir = "./data"

# Extract the contents of the zip file to the specified directory
zip_file.extractall(extract_dir)

# Close the zip file
zip_file.close()

In [13]:
dataset_samsum = load_from_disk('data/samsum_dataset')
dataset_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

# New Section

In [14]:
split_lengths = [len(dataset_samsum[split]) for split in dataset_samsum]

print(f"split_lengths : {split_lengths}")
print(f"features : {dataset_samsum['train'].column_names}")
print(f"\nDialogue:")

print(f"{dataset_samsum['test'][1]['dialogue']}")
print("\nSummary")
print(f"{dataset_samsum['test'][1]['summary']}")

split_lengths : [14732, 819, 818]
features : ['id', 'dialogue', 'summary']

Dialogue:
Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll check.
Eric: Sure.
Rob: Turns out no! There are some of his stand-ups on youtube.
Eric: Gr8! I'll watch them now!
Rob: Me too!
Eric: MACHINE!
Rob: MACHINE!
Eric: TTYL?
Rob: Sure :)

Summary
Eric and Rob are going to watch a stand-up on youtube.


In [16]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['dialogue'],max_length = 1024, truncation = True )

    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['summary'],max_length=128, truncation = True )

    return {
        'input_ids' :  input_encodings['input_ids'],
        'attention_mask' : input_encodings['attention_mask'],
        'labels' :target_encodings['input_ids']

    }



In [17]:
dataset_samsum_pt = dataset_samsum.map(convert_examples_to_features, batched = True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]c:\Users\admin\.conda\envs\nlp_text_env\lib\site-packages\transformers\tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 818/818 [00:00<00:00, 3736.17 examples/s]


In [18]:
dataset_samsum_pt = dataset_samsum.map(convert_examples_to_features, batched = True)

Map: 100%|██████████| 14732/14732 [00:02<00:00, 4922.23 examples/s]


In [19]:
dataset_samsum_pt['train']

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14732
})

In [20]:
# Training

from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

In [21]:
from transformers import TrainingArguments, Trainer

trainer_args = TrainingArguments(
    output_dir = 'pegasus-samsum',num_train_epochs=1, warmup_steps=500,
    per_device_train_batch_size=1, per_device_eval_batch_size=1,
    weight_decay=0.01, logging_steps=10,
    evaluation_strategy='steps',eval_steps=500,save_steps=1e6,
    gradient_accumulation_steps=16
)

c:\Users\admin\.conda\envs\nlp_text_env\lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [22]:
trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["test"],
                  eval_dataset=dataset_samsum_pt["validation"])

In [23]:
trainer.train()

  0%|          | 0/51 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [20]:
# Evaluation

def generate_batch_sized_chunks(list_of_elements, batch_size):
  """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
  for i in range(0, len(list_of_elements), batch_size):
      yield list_of_elements[i : i + batch_size]


def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                                batch_size=16, device =device,
                                column_text="article",
                                column_summary = "highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text],batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=1024, truncation=True,
                           padding="max_length",return_tensors = "pt")

        summaries = model.generate(input_ids = inputs["input_ids"].to(device),
                                   attention_mask = inputs["attention_mask"].to(device),
                                   length_penalty=0.8, num_beams=8, max_length=128)
        '''parameter for length penalty ensures that the model does not generate sequences. that are too long. '''

        # Finally, we decode the generated texts,
        # replace the token, and add the decoded texts with the references to the metric.

        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                              clean_up_tokenization_spaces = True)
                    for s in summaries]

        decoded_summaries = [d.replace(""," ") for d in decoded_summaries]


        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    # Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score


In [21]:
rouge_names = ["rouge1","rouge2","rougeL","rougeLsum"]
rouge_metric = load_metric('rouge')

<ipython-input-21-cc41bb0ac104>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric('rouge')
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [25]:
score = calculate_metric_on_test_ds(
    dataset_samsum['test'][0:10], rouge_metric,trainer.model, tokenizer, batch_size =2, column_text='dialogue', column_summary = 'summary')

rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)

pd.DataFrame(rouge_dict, index=[f'pegasus'])

100%|██████████| 5/5 [00:16<00:00,  3.28s/it]


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.022145,0.0,0.02181,0.022025


In [26]:
## save model
model_pegasus.save_pretrained("pegasus-samsum-model")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}


In [27]:
## save tokenizer
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [28]:
## Load
tokenizer = AutoTokenizer.from_pretrained("/content/tokenizer")

In [29]:
# Prediction
gen_kwargs = {"length_penalty":0.8, "num_beams":8, "max_length":128}


sample_text = dataset_samsum["test"][0]["dialogue"]

reference = dataset_samsum["test"][0]["summary"]

pipe = pipeline("summarization",model="pegasus-samsum-model",tokenizer=tokenizer)

##
print("Dialogue:")
print(sample_text)


print("\nReference Summary:")
print(reference)

print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

Your max_length is set to 128, but your input_length is only 122. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=61)


Dialogue:
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye

Reference Summary:
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.

Model Summary:
Amanda: Ask Larry Amanda: He called her last time we were at the park together .<n>Hannah: I'd rather you texted him .<n>Amanda: Just text him .
